# Tratamiento de datos para la generación de la tabla de características de un partido

En este notebook se realiza un tratamiento de los datos para poder generar la tabla de características a la hora de predecir un partido.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importamos archivo puntuaciones FIFA

punt_fifa = pd.read_excel('T1_Fifa_Puntuaciones.xlsx')

In [3]:
punt_fifa.head()

,Country,Punt_Ataque,Punt_Med,Punt_Def
0,ALEMANIA,85,85,85
1,ARABIA SAUDI,71,72,71
2,ARGENTINA,86,83,81
3,AUSTRALIA,69,72,71
4,AUSTRIA,80,77,75


In [4]:
# Creamos campo media de equipo

punt_fifa['Punt_Media_Eq'] = (punt_fifa['Punt_Ataque'] + punt_fifa['Punt_Med'] + punt_fifa['Punt_Def'])/3

In [5]:
punt_fifa.head()

,Country,Punt_Ataque,Punt_Med,Punt_Def,Punt_Media_Eq
0,ALEMANIA,85,85,85,85.000000
1,ARABIA SAUDI,71,72,71,71.333333
2,ARGENTINA,86,83,81,83.333333
3,AUSTRALIA,69,72,71,70.666667
4,AUSTRIA,80,77,75,77.333333


In [6]:
len(punt_fifa)

61

In [7]:
# Importamos tabla ranking FIFA

ranking_fifa = pd.read_excel('RANKING_FIFA.xlsx', sheetname='2018')

In [8]:
ranking_fifa.head()

,EQUIPO,RANKING_FIFA
0,ALEMANIA,1
1,BRASIL,2
2,BELGICA,3
3,PORTUGAL,4
4,ARGENTINA,5


In [9]:
# Cruzamos

df1 = punt_fifa.merge(ranking_fifa, left_on='Country', right_on='EQUIPO', how='inner')

In [10]:
len(df1)

61

In [11]:
df1.head()

,Country,Punt_Ataque,Punt_Med,Punt_Def,Punt_Media_Eq,EQUIPO,RANKING_FIFA
0,ALEMANIA,85,85,85,85.000000,ALEMANIA,1
1,ARABIA SAUDI,71,72,71,71.333333,ARABIA SAUDI,67
2,ARGENTINA,86,83,81,83.333333,ARGENTINA,5
3,AUSTRALIA,69,72,71,70.666667,AUSTRALIA,36
4,AUSTRIA,80,77,75,77.333333,AUSTRIA,26


In [139]:
df1.to_excel('df1.xlsx')

In [12]:
# Importamos la información de tranchas del ranking

tranchas = pd.read_excel('PORCEN_TRANCHAS.xlsx')

In [13]:
tranchas.head()

,EQUIPO,TRANCHA,-1,0,1,POR_VIC
0,ALEMANIA,1,2,3,1.0,0.166667
1,ALEMANIA,2,2,3,5.0,0.500000
2,ALEMANIA,3,2,1,9.0,0.750000
3,ALEMANIA,4,1,2,4.0,0.571429
4,ALEMANIA,5,0,0,0.0,0.000000


In [14]:
# Le pegamos el ranking

tranchas_2 = tranchas.merge(ranking_fifa, left_on='EQUIPO', right_on='EQUIPO', how='left')

In [15]:
tranchas_2.head()

,EQUIPO,TRANCHA,-1,0,1,POR_VIC,RANKING_FIFA
0,ALEMANIA,1,2,3,1.0,0.166667,1
1,ALEMANIA,2,2,3,5.0,0.500000,1
2,ALEMANIA,3,2,1,9.0,0.750000,1
3,ALEMANIA,4,1,2,4.0,0.571429,1
4,ALEMANIA,5,0,0,0.0,0.000000,1


In [142]:
tranchas_2.to_excel('tranchas_2.xlsx')

In [16]:
df1.head()

,Country,Punt_Ataque,Punt_Med,Punt_Def,Punt_Media_Eq,EQUIPO,RANKING_FIFA
0,ALEMANIA,85,85,85,85.000000,ALEMANIA,1
1,ARABIA SAUDI,71,72,71,71.333333,ARABIA SAUDI,67
2,ARGENTINA,86,83,81,83.333333,ARGENTINA,5
3,AUSTRALIA,69,72,71,70.666667,AUSTRALIA,36
4,AUSTRIA,80,77,75,77.333333,AUSTRIA,26


Creamos el código para un partido

In [26]:
# Función que me devuelve la trancha del ranking a la que pertenece una posición de ranking

def trancha_ranking(ranking):
    if ranking <= 10:
        trancha = 1
    elif ranking <= 20:
        trancha = 2
    elif ranking <= 50:
        trancha = 3
    elif ranking <= 100:
        trancha = 4
    else:
        trancha = 5
    return trancha

In [27]:
partido = ['ALEMANIA', 'ARABIA SAUDI']

In [28]:
local = df1[df1['Country'] == partido[0]]
visitante = df1[df1['Country'] == partido[1]]

union = pd.DataFrame()

union['DIF_AT_DEF'] = pd.Series(int(local['Punt_Ataque']) - int(visitante['Punt_Def']))
union['DIF_MED'] = pd.Series(int(local['Punt_Med']) - int(visitante['Punt_Med']))
union['DIF_DEF_AT'] = pd.Series(int(local['Punt_Def']) - int(visitante['Punt_Ataque']))
union['DIF_RANK'] = pd.Series(int(local['RANKING_FIFA']) - int(visitante['RANKING_FIFA']))

In [29]:
union

,DIF_AT_DEF,DIF_MED,DIF_DEF_AT,DIF_RANK
0,14,13,14,-66


In [30]:
ranking_local = int(tranchas_2[tranchas_2['EQUIPO'] == partido[0]]['RANKING_FIFA'][:1])
trancha_local = trancha_ranking(ranking_local)

ranking_visitante = int(tranchas_2[tranchas_2['EQUIPO'] == partido[1]]['RANKING_FIFA'][:1])
trancha_visitante = trancha_ranking(ranking_visitante)

In [31]:
cond1 = tranchas_2['EQUIPO'] == partido[0]
cond2 = tranchas_2['TRANCHA'] == trancha_visitante

porc_local = float(tranchas_2[cond1 & cond2]['POR_VIC'])

cond3 = tranchas_2['EQUIPO'] == partido[1]
cond4 = tranchas_2['TRANCHA'] == trancha_local

porc_visitante = float(tranchas_2[cond3 & cond4]['POR_VIC'])


In [32]:
porc_local

0.5714285714285714

In [33]:
union['DIF_VD_TRAN'] = pd.Series(porc_local - porc_visitante)


In [34]:
union

,DIF_AT_DEF,DIF_MED,DIF_DEF_AT,DIF_RANK,DIF_VD_TRAN
0,14,13,14,-66,0.571429


Generalizamos metiendo el código en una función. Para dicha función necesitaremos las tablas:
* `df1`
* `tranchas_2`

In [35]:
# Necesito las tablas df1 y tranchas_2

def genera_tabla_caracteristicas(partido):
    
    local = df1[df1['Country'] == partido[0]]
    visitante = df1[df1['Country'] == partido[1]]

    union = pd.DataFrame()

    union['DIF_AT_DEF'] = pd.Series(int(local['Punt_Ataque']) - int(visitante['Punt_Def']))
    union['DIF_MED'] = pd.Series(int(local['Punt_Med']) - int(visitante['Punt_Med']))
    union['DIF_DEF_AT'] = pd.Series(int(local['Punt_Def']) - int(visitante['Punt_Ataque']))
    union['DIF_RANK'] = pd.Series(int(visitante['RANKING_FIFA']) - int(local['RANKING_FIFA']))
    
    ranking_local = int(tranchas_2[tranchas_2['EQUIPO'] == partido[0]]['RANKING_FIFA'][:1])
    trancha_local = trancha_ranking(ranking_local)

    ranking_visitante = int(tranchas_2[tranchas_2['EQUIPO'] == partido[1]]['RANKING_FIFA'][:1])
    trancha_visitante = trancha_ranking(ranking_visitante)
    
    cond1 = tranchas_2['EQUIPO'] == partido[0]
    cond2 = tranchas_2['TRANCHA'] == trancha_visitante

    porc_local = float(tranchas_2[cond1 & cond2]['POR_VIC'])

    cond3 = tranchas_2['EQUIPO'] == partido[1]
    cond4 = tranchas_2['TRANCHA'] == trancha_local

    porc_visitante = float(tranchas_2[cond3 & cond4]['POR_VIC'])

    union['DIF_VD_TRAN'] = pd.Series(porc_local - porc_visitante)

    return union

In [36]:
genera_tabla_caracteristicas(['ALEMANIA', 'IRLANDA DEL NORTE'])

,DIF_AT_DEF,DIF_MED,DIF_DEF_AT,DIF_RANK,DIF_VD_TRAN
0,14,14,14,28,0.75
